<a href="https://colab.research.google.com/github/paulcodrea/reddit_humor/blob/main/2b_tf_idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM with a tf-idf embedding vector 

In [82]:
import csv
import nltk
from nltk import word_tokenize, RegexpTokenizer
nltk.download('stopwords')
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))
import numpy as np
import pandas as pd
from nltk.stem.porter import *
stemmer = PorterStemmer()
import math
import pickle


nltk.download('punkt')

from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout 
from keras.layers import LSTM, Embedding, TextVectorization

from sklearn.model_selection import train_test_split

from nltk.tokenize import word_tokenize

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.metrics import precision_score, recall_score, f1_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [83]:
config = {
    "learning_rate": 0.001,
    "epochs": 3, 
    "batch_size": 70,
    "test_p": 0.2,
    "val_p": 0.1,
    "LSTM_layer": [50, 100],
    "Dropout_layer": [0.15, 0.2],
    "activation": 'sigmoid',
    ##################### SAVE FOR LIVE DEMO #############################
    "model_path": './model/2b_model.h5',
    "tokenizer_path": './model/2b_tokenizer.pickle',
    "data_path": "model/2b_data.csv",
}

# Methods:

In [84]:
class LSTM_model:
  def __init__(self, path):
    self.path = path # Path to the dataset
    self.data = pd.DataFrame() # Dataframe to store the dataset
    self.max_joke_size = 0
    self.vocabulary_size = 0
    self.tokenizer = Tokenizer(num_words=None, split=' ') # maybe delete

    self.list_jokes_tokenized = []
    self.model = None

  def read_dataset(self):
    """
    """
    self.data = pd.read_csv(self.path)

    print("Data successfuly read!")

    print("This is the first line from the data file: \n", self.data.head(1))

  def tokenise_clean_text(self):
    """
    Tokenises the data from the CSV file.
    """
    arr = []
    for line in self.data['clean_text']:
      arr.append(line.split(' '))
    self.list_jokes_tokenized = [x[:(len(x) - 1)] for x in arr]
    print('Tokenisation is complete!')
    print("This is the first item from the tokenised list of jokes:\n", self.list_jokes_tokenized[0])
    print("This is how many jokes the data set has:\n",len(self.list_jokes_tokenized))


  def return_vocab(self) -> list:
    """
    Retuns the vocabulary of a document.
    """
    computed_vocab = []
    for line in self.list_jokes_tokenized:
      for word in line:
        computed_vocab.append(word)
    computed_vocab = list(dict.fromkeys(computed_vocab))
    print("The vocabulary has been created!")
    print("This is the len of the vocabulary:\n ", len(computed_vocab))

    print("This is the first token from the corpus, pre-processed: \n", computed_vocab[0])
    self.vocabulary_size = len(computed_vocab)

    return computed_vocab

  def term_frequency_corpus(self, vocabulary):
    """
    Returns the a dictionary where the keys are the vocabulary words and the values
    are the frequency of the word in the whole corpus.
    """
    vocab_freq = {}
    for word in vocabulary:
      vocab_freq[word] = 0
    for joke in self.list_jokes_tokenized:
      for token in joke:
        vocab_freq[token] = vocab_freq[token] + 1
    print("The vocabulary with the associated frequency for each word has been generated!")
    print("This is how many times the word 'joke' is in our corpus\n", vocab_freq["joke"])
    
    return vocab_freq

  def term_frequency(self, vocab: dict):
    """
    Returns a dictionary where the keys are the vocabulary words and the values are
    a list that represents the frequency of the word in each joke divided by the 
    length of the joke.
    """
    dict_term_freq = {}
    for word in vocab.keys():
      dict_term_freq[word] = []
    for word in vocab.keys():
      for joke in self.list_jokes_tokenized:
        freq = 0
        for word_index in range(len(joke)):
          if word == joke[word_index]:
            freq = freq + 1
        dict_term_freq[word].append(freq / len(joke))
    print("Term frequency matrix/dictionary was generated!")
    print("This is the term frequency vector for word 'joke':\n", dict_term_freq["joke"])

    return dict_term_freq

  def document_frequency(self, vocabulary: dict):
    """
    Returns a dictionary where the keys are all words in the vocabulary and the 
    values the list of jokes the word appears in.
    """
    doc_freq = {}
    for word in vocabulary.keys():
      doc_freq[word] = []
      for joke_index in range(len(self.list_jokes_tokenized)):
        if word in self.list_jokes_tokenized[joke_index]:
          doc_freq[word].append(joke_index)
    print("A list of all documents in which the words from our vocab appear in have been generated!")
    print("The list of all documents that contain the word 'Joke' looks like this:\n",doc_freq["joke"] )

    return doc_freq

  def generate_idf(self, document_frequencies):
    idf = {}
    for word in document_frequencies.keys():
      idf[word] = math.log(len(self.list_jokes_tokenized)/len(document_frequencies[word]))
    print("A dictionary with each word's tf-idf values has been generated!")
    return idf

  def generate_tf_idf(self, vocab_frequencies, tf, idf):
    dict_tf_idf = {}

    for word in vocab_frequencies.keys():
      dict_tf_idf[word] = []
      for tf_index in range(len(tf[word])):
        dict_tf_idf[word].append(tf[word][tf_index] * idf[word])

    print("The tf-idf values have beed generated! ")
    print("this is how the tf-idf vector looks like for the word 'joke':\n",dict_tf_idf["joke"])
    print("This is the length of each tf-idf vector. Each value represent's the word's tf-idf for each sentence:\n", len(dict_tf_idf["joke"]))
    return dict_tf_idf

  def joke_as_tf_idf(self, dict_tf_idf):
    jokes_as_tf_idf = []

    for joke_index in range(len(self.list_jokes_tokenized)):
      joke = self.list_jokes_tokenized[joke_index]
      tf_idf_list = []
      for word in joke:
        tf_idf_list.append(dict_tf_idf[word][joke_index])
      tf_idf = np.asarray(tf_idf_list)
      jokes_as_tf_idf.append(tf_idf)

      # calculate max_joke size to use in the padding step
      if len(joke) > self.max_joke_size:
        self.max_joke_size = len(tf_idf_list)
        
    jokes_as_tf_idf = np.asarray(jokes_as_tf_idf)

    print("The jokes have been embedded by their tf-idf values!")
    print("This is the max joke length for our corpus: ", self.max_joke_size)
    print("This is how the first joke looks like before the embedding step: \n", self.list_jokes_tokenized[0])
    print("This is how the tf-idf embedding of the first joke looks like:\n",jokes_as_tf_idf[0])

    return jokes_as_tf_idf

  def pad_jokes(self, jokes_as_tf_idf_np):
    new_arr = np.zeros((len(jokes_as_tf_idf_np), self.max_joke_size))
    old_arr = jokes_as_tf_idf_np
    for idx, joke in enumerate(jokes_as_tf_idf_np):
      len_joke = len(joke)
      joke_x = jokes_as_tf_idf_np[idx]
      new_arr[idx] = np.append([0] * (self.max_joke_size - len_joke), joke)

    print("This is the shape of the input:/n", new_arr.shape)
    jokes_as_tf_idf_np = new_arr
    return jokes_as_tf_idf_np

  def generate_train_test_data(self, jokes_list):
    train_max_index = int(len(jokes_list) * 0.8)
    X_train = np.zeros(shape=(train_max_index, self.max_joke_size))
    X_test = np.zeros(shape=(len(jokes_list) - train_max_index, self.max_joke_size))
    index_returned_data = 0
    for index in range(0, train_max_index):
      X_train[index_returned_data] = jokes_list[index]
      index_returned_data = index_returned_data + 1
    y_train = self.data["funny"][:train_max_index]
    index_returned_data = 0
    for index in range(train_max_index, len(jokes_list)):
      X_test[index_returned_data] = jokes_list[index]
      index_returned_data = index_returned_data + 1
    y_test = self.data["funny"][train_max_index:]
    
    return X_train, X_test, y_train, y_test


  def LSTM_model(self, X_train, X_test, y_train, y_test):
    """
    Splits the data into train and validation sets.

    Constructs the LSTM model.
    """

    # print("X shape: ", X.shape)
    # print("y shape: ", y.shape)

    # split the data into train and validation sets and make them random
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=config['test_p'], random_state=42)

    print("X_train:", len(X_train))
    print("X_test:", len(X_test))
    print("y_train:", len(y_train))
    print("y_test:", len(y_test))

    print(X_train.shape)
    print(y_train.shape)

    model = Sequential()
    model.add(Embedding(input_dim=self.vocabulary_size, output_dim=self.max_joke_size, input_length=int(X_train.shape[1])))
    # model.add(Dropout(config['Dropout_layer'][0]))
    model.add(LSTM(50, return_sequences=True))
    model.add(LSTM(10))

    model.add(Dropout(0.5))
    model.add(Dense(units=1, activation=config['activation']))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=config['epochs'], batch_size=config['batch_size'], verbose='auto', validation_split=config['val_p'])

    # Evaluate the model
    scores = model.evaluate(X_test, y_test)
    print("Accuracy: %.2f%%" % (scores[1] * 100))

    # Print Precision and Recall
    y_pred = model.predict(X_test)
    y_pred = np.round(y_pred)
    precision = precision_score(y_test, y_pred) * 100
    recall = recall_score(y_test, y_pred) * 100
    f1 = f1_score(y_test, y_pred) * 100
    print("Precision: %.2f%%" % (precision))
    print("Recall: %.2f%%" % (recall))
    print("F1-Score: %.2f%%" % (f1))

    # self.save_data(scores[1], precision, recall, f1)


  def save_data(self, accuracy, precision, recall, f1):
    """
    Saves the data.
    """
    # Add in dataframe master_df max_len, accuracy, precision, recall, f1-score
    ret = pd.DataFrame(columns=['max_len', 'accuracy', 'precision', 'recall', 'f1-score'])
    ret.loc[0] = [self.max_joke_size, accuracy, precision, recall, f1]
    ret.to_csv(config['data_path'])
    
    # self.model.save(config['model_path']) # save the model

    # save the tokenizer
    # with open(config['tokenizer_path'], 'wb') as handle:
    #     pickle.dump(self.tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Loading data from the dataset:

In [85]:
path = 'drive/MyDrive/Humour_Detection_Dataset/final_jokes(2918).csv'

joke_model = LSTM_model(path)
joke_model.read_dataset()

Data successfuly read!
This is the first line from the data file: 
    Unnamed: 0                                     joke  \
0           0  What vocal group always kills? The OJs.   

                                              tokens  token_count  funny  \
0  ['What', 'vocal', 'group', 'always', 'kills', ...            7      0   

                               clean_text  
0  what vocal group always kills the ojs   


# Pre-Processing

The functions below enable the data from the csv to be transformed in variables used to compute the tf and idf.

In [86]:
joke_model.tokenise_clean_text()
vocabulary = joke_model.return_vocab()

Tokenisation is complete!
This is the first item from the tokenised list of jokes:
 ['what', 'vocal', 'group', 'always', 'kills', 'the', 'ojs']
This is how many jokes the data set has:
 2918
The vocabulary has been created!
This is the len of the vocabulary:
  5778
This is the first token from the corpus, pre-processed: 
 what


# Computing Term Frequency (TF)

Create TF for each word in the vocabulary:

The number of times a word appears in a document divded by the total number of words in the document. Every document has its own term frequency.

In [87]:
# TF
# the frequency of each word from the vocabulary in the corpus 
vocab_freq = joke_model.term_frequency_corpus(vocabulary)
# Calculate tf number of times a word appears in a doc / total number of words in the doc
tf = joke_model.term_frequency(vocab_freq)

The vocabulary with the associated frequency for each word has been generated!
This is how many times the word 'joke' is in our corpus
 127
Term frequency matrix/dictionary was generated!
This is the term frequency vector for word 'joke':
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

# Computing the Inverse Document Frequency (IDF)

Calculating IDF for each word:

The log of the number of documents divided by the number of documents that contain the word w. The IDF of a word is computed once for all documents.

In [88]:
# IDF
# create a list of all the docs in which the vocab words occur -> to be used in idf
dict_document_freq = joke_model.document_frequency(vocab_freq)
idf = joke_model.generate_idf(dict_document_freq)

A list of all documents in which the words from our vocab appear in have been generated!
The list of all documents that contain the word 'Joke' looks like this:
 [38, 50, 91, 135, 193, 196, 218, 219, 228, 286, 315, 329, 339, 354, 381, 393, 440, 533, 564, 592, 699, 704, 709, 712, 715, 742, 793, 796, 805, 823, 858, 860, 866, 877, 929, 961, 984, 999, 1082, 1115, 1132, 1150, 1153, 1164, 1193, 1217, 1235, 1264, 1265, 1280, 1326, 1389, 1412, 1414, 1434, 1509, 1512, 1513, 1523, 1588, 1608, 1661, 1668, 1669, 1701, 1719, 1724, 1787, 1854, 1864, 1909, 1918, 1924, 1929, 1961, 1997, 2015, 2027, 2029, 2042, 2074, 2080, 2103, 2181, 2190, 2380, 2381, 2428, 2434, 2441, 2446, 2476, 2492, 2513, 2543, 2684, 2716, 2717, 2735, 2751, 2756, 2759, 2762, 2783, 2820, 2844, 2861, 2879, 2888]
A dictionary with each word's tf-idf values has been generated!


# Computing the TF-IDF vectors




TF-IDF is simply the TF multiplied by IDF.

The output will be a list of tf-idf values (word x joke)

In [89]:
# TF IDF
dict_tf_idf = joke_model.generate_tf_idf(vocab_freq, tf, idf)

The tf-idf values have beed generated! 
this is how the tf-idf vector looks like for the word 'joke':
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6574611693707175, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8218264617133968, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16436529234267938, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.6436529234267936, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

Representing each joke by an embedding vector.


In [90]:
# embed
jokes_as_tf_idf_np = joke_model.joke_as_tf_idf(dict_tf_idf)

The jokes have been embedded by their tf-idf values!
This is the max joke length for our corpus:  134
This is how the first joke looks like before the embedding step: 
 ['what', 'vocal', 'group', 'always', 'kills', 'the', 'ojs']
This is how the tf-idf embedding of the first joke looks like:
 [0.18756714 1.04078665 0.79725121 0.55021705 0.8427446  0.38434096
 1.04078665]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:139: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


# Creating training and testing datasets

Pad the jokes. 



In [91]:
# pad
jokes_as_tf_idf_np = joke_model.pad_jokes(jokes_as_tf_idf_np)

This is the shape of the input:/n (2918, 134)


Split the data in training - 80% and testing - 20%.

In [92]:
X_train, X_test, y_train, y_test = joke_model.generate_train_test_data(jokes_as_tf_idf_np)

# Training and saving the model

In [93]:
joke_model.LSTM_model(X_train, X_test, y_train, y_test)

X_train: 2334
X_test: 584
y_train: 2334
y_test: 584
(2334, 134)
(2334,)
Epoch 1/3
30/30 [==============================] - 8s 167ms/step - loss: 0.6935 - accuracy: 0.4967 - val_loss: 0.6916 - val_accuracy: 0.5256
Epoch 2/3
30/30 [==============================] - 4s 136ms/step - loss: 0.6913 - accuracy: 0.5229 - val_loss: 0.6913 - val_accuracy: 0.5641
Epoch 3/3
19/19 [==============================] - 0s 24ms/step - loss: 0.6876 - accuracy: 0.6045
Accuracy: 60.45%
Precision: 61.29%
Recall: 67.86%
F1-Score: 64.41%
